# INTA database inspection

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys

import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import pprint

detector_path = '..'
sys.path.append(detector_path)

from sleep.inta import INTA
from sleep.inta import KEY_EEG, KEY_PAGES, KEY_ID, KEY_MARKS
from sleep.data_ops import seq2inter, inter2seq, seq2inter_with_pages
from evaluation.metrics import by_sample_confusion, by_sample_iou, by_event_confusion, matching

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
dataset = INTA(load_checkpoint=True)

## Checking general numbers

In [ ]:
print('Sample frequency:', dataset.fs)
print('Page duration and size:', dataset.page_duration, dataset.page_size)
print('Registers ids:', dataset.all_ids)
print('Number of loaded registers:', len(dataset.data))
print('')
print('Train set IDs:', dataset.train_ids)
print('Test set IDs:', dataset.test_ids)

## Check one subject

In [ ]:
def plot_subject_global(signal, fs, pages, page_size, marks_1):
    time_axis = np.arange(signal.size) / fs
    max_time = time_axis[-1]
    
    n_pages = np.ceil(signal.size / page_size).astype(int)
    n2_stages = np.zeros(n_pages, dtype=np.uint8)
    for page in pages:
        n2_stages[page] = 1
    
    marks_1_in_page = np.zeros(n_pages, dtype=np.int32)
    marks_1_n2_without_marks = 0
    for page in pages:
        start_sample = page * page_size
        end_sample = (page+1) * page_size
        marks_1_presence = np.sum(marks_1[start_sample:end_sample])
        if marks_1_presence == 0:
            marks_1_n2_without_marks += 1
        marks_1_in_page[page] = marks_1_presence
    
    print('Number of N2 pages without E1 marks: %d' % marks_1_n2_without_marks)
    
    fig, ax = plt.subplots(3, 1, figsize=(15, 5))
    
    ax[0].set_title('Normalized EEG Signal')
    ax[0].plot(time_axis, signal)
    ax[0].set_xlabel('Time [s]')
    ax[0].set_xlim([0, max_time])

    ax[1].set_title('N2 stages (white is active)')
    ax[1].imshow(n2_stages[np.newaxis, :], interpolation=None, aspect='auto', cmap='gray', extent=[0, max_time, 0, 1])
    ax[1].axis('off')
    
    ax[2].set_title('E1 marks presence (white is active)')
    ax[2].imshow(marks_1_in_page[np.newaxis, :], interpolation=None, aspect='auto', extent=[0, max_time, 0, 1], cmap='gray')
    ax[2].axis('off')
    
    plt.tight_layout()
    plt.show()

### Complete register visualization

In [ ]:
# Registers ids: [1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 17, 19]
subject_id = 2

In [ ]:
ind_idx = dataset.all_ids.index(subject_id)
ind_dict = dataset.data[ind_idx]
subject_id = ind_dict[KEY_ID]
signal = ind_dict[KEY_EEG]
pages = ind_dict[KEY_PAGES]
marks_1 = ind_dict['%s_1' % KEY_MARKS]

print('Loaded subject ID %s' % subject_id)
print('Signal size %d. Number of pages %d' % (signal.shape[0], pages.shape[0]))

In [ ]:
plot_subject_global(signal, dataset.fs, pages, dataset.page_size, marks_1)

## Marks statistics from saved data

In [ ]:
marks_1_stamps = seq2inter(marks_1)
print('Number of marks from E1: %d' % marks_1_stamps.shape[0])

In [ ]:
durations_1 = np.diff(marks_1_stamps)[:, 0] / dataset.fs

fig, ax = plt.subplots(1, 1)
ax.hist(durations_1, bins=20)
ax.set_title('E1 marks durations')
ax.set_xlabel('Duration [s]')
plt.show()

## Page visualization

In [ ]:
n2_signal, n2_marks_1 = dataset.get_subject_data(subject_id, which_expert=1, verbose=True)
pages = dataset.get_subject_pages(subject_id)

In [ ]:
print('Number of n2 pages:', n2_signal.shape)
tmp = np.sum(n2_marks_1, axis=1)
tmp = sum((tmp==0).astype(int))
print('Number of pages without E1 marks:', tmp)

In [ ]:
n2_marks_1_stamps = seq2inter_with_pages(n2_marks_1, pages)
print('Number of E1 marks:', n2_marks_1_stamps.shape)

In [ ]:
def plot_page(page, signal, e1_marks, n2_pages, fs):
    fig = plt.figure(figsize=(20, 5)) 
    gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1]) 
    
    page = page - 1
    
    segment_signal = signal[page, :]
    segment_e1_marks = e1_marks[page, :]
    time_axis = np.arange(signal.shape[1])/fs
    
    # Signal
    ax0 = fig.add_subplot(gs[0])
    ax0.plot(time_axis, segment_signal)
    ax0.set_yticks([])
    ax0.set_xlim([0, 20])
    ax0.set_ylim([-6, 6])
    ax0.set_title('EEG Signal, F4-C4 (intervals of 0.5s are shown), original page %d' % n2_pages[page])
    ax0.set_xticks([0, 5, 10, 15, 20])
    ax0.set_xticks(np.arange(0, 20, 0.5), minor=True)
    ax0.grid(b=True, axis='x', which='minor')
    
    # Expert mark
    ax2 = fig.add_subplot(gs[1])
    ax2.imshow(segment_e1_marks[np.newaxis, :], interpolation=None, aspect='auto', cmap='gray')
    ax2.axis('off')
    ax2.set_title('Expert 1 (white is active)')
    
    plt.tight_layout()
    plt.show()

In [ ]:
widgets.interact(
    lambda page: plot_page(page, n2_signal, n2_marks_1, pages, dataset.fs),
    page=widgets.IntSlider(min=1,max=n2_signal.shape[0],step=1,value=1, continuous_update=False));